In [1]:
from WikiXmlHandler import WikiXmlHandler
import xml.sax
import mwparserfromhell
import stanfordnlp

import subprocess
import pandas as pd
import os
import Utils

from ProcessArticle import ProcessArticle

In [2]:
infoboxes = pd.read_csv("./saved/infobox-list.csv")["infobox template name"].values.tolist()
out_file = "./saved/debug.csv"

func = ProcessArticle(infoboxes)

In [3]:
%%time
file = "./test_data/test.xml"

handler = WikiXmlHandler(out_file, func, 1000)
handler._DEBUG = True

parser = xml.sax.make_parser()
parser.setContentHandler(handler)

for line in open(file, "r", encoding="utf8"):
    parser.feed(line)

# handler.write_df()

Wall time: 784 ms


In [4]:
n_df = pd.read_csv(out_file, delimiter="\t")

In [5]:
n_df

,title,type,def words,infobox_name,infobox,categories,wikilinks
0,Abraham Lincoln,article,-,Infobox officeholder,name:Abraham Lincoln|image:Abraham Lincoln O-7...,Abraham Lincoln|1809 births|1865 deaths|1865 m...,President of the United States|Hannibal Hamlin...
1,Apollo,article,-,Infobox deity,type:Greek|name:Apollo|image:File:Apollo of th...,Apollo|Arts gods|Deities in the Iliad|Dragonsl...,Apollo Belvedere|Mount Olympus|Lyre|laurel wre...
2,Apollo program,article,-,Infobox space program,name:Apollo program|image:frameless|upright|al...,Human spaceflight programs|Apollo program|Spac...,File:Apollo program.svg|NASA|lunar landing|Sat...


In [4]:
n_df[n_df.title.str.contains("(?i)cobol")]

,title,type,def words,infobox_name,infobox,categories,wikilinks


In [ ]:
for i, a in enumerate(handler._data):
    if a[0] == "List of artificial intelligence projects":
        print("{}: {}".format(i, a[0]))

In [ ]:
for i, a in enumerate(handler._data):
    # if a[0] == "COBOL":
    print("{}: {}".format(i, a[0]))

In [4]:
page = handler._data[0]
text = handler._text[0]

In [ ]:
page[4].split("|")

In [5]:
infoboxes = pd.read_csv("./saved/infobox-list.csv")["infobox template name"].values.tolist()
cols = ["title", "type", "def words", "infobox_name", "infobox", "categories","wikilinks"]

In [6]:
wiki = mwparserfromhell.parse(text)

In [7]:
a = wiki.strip_code().split("\n")
a = [" ".join(Utils.cleanhtml(x.strip()).split()) for x in a if not x.isspace() and x != ""]
a = a[0] # 1st paragraph
a

'Abraham Lincoln (; February 12, 1809 – April 15, 1865) was an American statesman and lawyer who served as the 16th president of the United States from March 1861 until his assassination in April 1865. Lincoln led the nation through the American Civil War, its bloodiest war and its greatest moral, constitutional, and political crisis. He preserved the Union, abolished slavery, strengthened the federal government, and modernized the U.S. economy.'

In [8]:
nlp = stanfordnlp.Pipeline(models_dir="./stanfordnlp_resources/", use_gpu=False);

Use device: cpu
---
Loading: tokenize
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': './stanfordnlp_resources/en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_parser.pt', 'pretrain_path': './stanfordnlp_resources/en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': '

In [9]:
doc = nlp(a)

..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


In [38]:
sentence = doc.sentences[0]

In [41]:
sentence.words[14]

<Word index=15;text=was;lemma=be;upos=AUX;xpos=VBD;feats=Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin;governor=18;dependency_relation=cop>

In [61]:
import datetime

months_choices = list()
for i in range(1,13):
    months_choices.append((i, datetime.date(2008, i, 1).strftime('%B')))

months_choices

[(1, 'January'),
 (2, 'February'),
 (3, 'March'),
 (4, 'April'),
 (5, 'May'),
 (6, 'June'),
 (7, 'July'),
 (8, 'August'),
 (9, 'September'),
 (10, 'October'),
 (11, 'November'),
 (12, 'December')]

In [64]:
import datetime

definitions = []
no_no = ["punct", "nummod", "appos", "det", "nmod:poss", "cop", "cc", "nsubj", "case"]
months = [datetime.date(2008, i, 1).strftime("%B").lower() for i in range(1,13)]

for word in sentence.words:
    index = word.index.rjust(2)
    word_lemma = word.lemma
    governor = sentence.words[word.governor-1].lemma if word.governor > 0 else "root"
    deprel = word.dependency_relation

    if deprel in no_no or word_lemma.lower() in months:
        continue

    print("{}\t{}\t\t\t{}\t\t\t{}".format(index, word_lemma, governor, deprel))

2	Lincoln			Abraham			flat
17	american			statesman			amod
18	statesman			root			root
20	lawyer			statesman			conj
22	serve			lawyer			acl:relcl
25	16th			president			amod
26	president			serve			obl
29	United			States			compound
30	States			president			nmod
36	assassination			serve			obl


In [45]:
# POS tags
print(" ". join(["{} ({})".format(word.text, word.upos) for word in sentence.words]))

Abraham (PROPN) Lincoln (PROPN) ( (PUNCT) ; (PUNCT) February (PROPN) 12 (NUM) , (PUNCT) 1809 (NUM) – (PUNCT) April (PROPN) 15 (NUM) , (PUNCT) 1865 (NUM) ) (PUNCT) was (AUX) an (DET) American (ADJ) statesman (NOUN) and (CCONJ) lawyer (NOUN) who (PRON) served (VERB) as (ADP) the (DET) 16th (ADJ) president (NOUN) of (ADP) the (DET) United (PROPN) States (PROPN) from (ADP) March (PROPN) 1861 (NUM) until (ADP) his (PRON) assassination (NOUN) in (ADP) April (PROPN) 1865 (NUM) . (PUNCT)


In [22]:
for word in sentence.tokens[14].words:
    if word.lemma in ["be"]:
        print("a")

a


In [23]:
sentence.tokens[14].text

'was'

In [37]:
print(*[f"index: {word.index.rjust(2)}\tword: {word.text.ljust(11)}\tgovernor index: {word.governor}\tgovernor: {(sentence.words[word.governor-1].text if word.governor > 0 else 'root').ljust(11)}\tdeprel: {word.dependency_relation}" for word in sentence.words], sep='\n')

index:  1	word: Lincoln    	governor index: 2	governor: led        	deprel: nsubj
index:  2	word: led        	governor index: 0	governor: root       	deprel: root
index:  3	word: the        	governor index: 4	governor: nation     	deprel: det
index:  4	word: nation     	governor index: 2	governor: led        	deprel: obj
index:  5	word: through    	governor index: 9	governor: War        	deprel: case
index:  6	word: the        	governor index: 9	governor: War        	deprel: det
index:  7	word: American   	governor index: 9	governor: War        	deprel: compound
index:  8	word: Civil      	governor index: 9	governor: War        	deprel: compound
index:  9	word: War        	governor index: 2	governor: led        	deprel: obl
index: 10	word: ,          	governor index: 9	governor: War        	deprel: punct
index: 11	word: its        	governor index: 13	governor: war        	deprel: nmod:poss
index: 12	word: bloodiest  	governor index: 13	governor: war        	deprel: amod
index: 13	word:

In [46]:
print(" ". join(["{} ({} - {})".format(word.text, word.upos, word.feats) for word in sentence.words]))

Abraham (PROPN - Number=Sing) Lincoln (PROPN - Number=Sing) ( (PUNCT - _) ; (PUNCT - _) February (PROPN - Number=Sing) 12 (NUM - NumType=Card) , (PUNCT - _) 1809 (NUM - NumType=Card) – (PUNCT - _) April (PROPN - Number=Sing) 15 (NUM - NumType=Card) , (PUNCT - _) 1865 (NUM - NumType=Card) ) (PUNCT - _) was (AUX - Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin) an (DET - Definite=Ind|PronType=Art) American (ADJ - Degree=Pos) statesman (NOUN - Number=Sing) and (CCONJ - _) lawyer (NOUN - Number=Sing) who (PRON - PronType=Rel) served (VERB - Mood=Ind|Tense=Past|VerbForm=Fin) as (ADP - _) the (DET - Definite=Def|PronType=Art) 16th (ADJ - Degree=Pos|NumType=Ord) president (NOUN - Number=Sing) of (ADP - _) the (DET - Definite=Def|PronType=Art) United (PROPN - Number=Sing) States (PROPN - Number=Sing) from (ADP - _) March (PROPN - Number=Sing) 1861 (NUM - NumType=Card) until (ADP - _) his (PRON - Gender=Masc|Number=Sing|Person=3|Poss=Yes|PronType=Prs) assassination (NOUN - Number=Sing) i

In [30]:
sentence.build_dependencies()

In [34]:
sentence.dependencies[0]

(<Word index=18;text=statesman;lemma=statesman;upos=NOUN;xpos=NN;feats=Number=Sing;governor=0;dependency_relation=root>,
 'nsubj',
 <Word index=1;text=Abraham;lemma=Abraham;upos=PROPN;xpos=NNP;feats=Number=Sing;governor=18;dependency_relation=nsubj>)

In [ ]:
wiki.filter_templates(matches="softredirect")

In [ ]:
"list of" in wiki.filter_templates(matches="DEFAULTSORT")[0].name.lower()

In [ ]:
# Creating dict
d = {col: [] for col in cols}
df = pd.DataFrame(columns=cols)
# parse page
# wiki = mwparserfromhell.parse(page[1])

In [ ]:
t = [None, "3", "None", "6", None, None,"1"]

In [ ]:
[d[cols[i]].append(t[i]) for i in range(len(t))]

In [ ]:
d

In [ ]:
df

In [ ]:
d

In [ ]:
# page title
# d["title"].append(page[0])
df["title"] = [page[0]]

In [ ]:
# defining article type
if page[2] != None:
    # d["type"].append("redirect")
    df["type"] = ["redirect"]
else:
    # d["type"].append("article")
    df["type"] = ["article"]

In [ ]:
# Getting article categories
# d["categories"].append(
#     "|".join([
#         list(filter(None, x.title.split(":")))[-1]
#         for x in wiki.filter_wikilinks(matches="category")
#     ]))
df["categories"] = ["|".join([
        list(filter(None, x.title.split(":")))[-1]
        for x in wiki.filter_wikilinks(matches="category")
    ])]

In [ ]:
# wikilinks, removing links to Category items
# d["wikilinks"].append("|".join(
#     [link.title.strip_code().strip() for link in wiki.filter_wikilinks() if "Category:" not in link.title.strip_code().strip()]))

df["wikilinks"] = ["|".join(
    [link.title.strip_code().strip() for link in wiki.filter_wikilinks() if "Category:" not in link.title.strip_code().strip()])]

# external links
# df["ext_links"] = "|".join([link.url.strip_code().strip() for link in wiki.filter_external_links()])

In [ ]:
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

In [ ]:
# search for infobox template name
box = None
for template in wiki.filter_templates():
    if template.name.strip() in infoboxes:
        box = wiki.filter_templates(matches=template.name.strip())[0]
        # d["infobox_name"].append(template.name.strip())
        df["infobox_name"] = [template.name.strip()]
        break

a = "|".join(["{}:{}".format(param.name.strip_code().strip(),
                            param.value.strip_code(keep_template_params=True).strip()) for param in box.params])
# d["infobox"].append(cleanhtml(a))
df["infobox"] = [cleanhtml(a)]

In [ ]:
df

In [ ]:
df["infobox"][0].split("|")

In [ ]:
df["wikilinks"][0].split("|")

In [ ]:
# mais proximo da solucao definitiva
for param in box.params:
    for x in param.value.filter_templates():
        print([y.value.strip_code() for y in x.params])

In [ ]:
# box.params.strip_code()
for param in box.params:
    # print(param.value.filter_text())
    # print(param.value.filter_wikilinks())
    print(param.value.strip_code(keep_template_params=True).strip())

In [ ]:
df["infobox"][0]